# DT Assignment

# Data Loading

In [3]:
import pandas as pd 
import numpy as np

In [4]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [25]:
def get_gini(df, label):
    # 총 데이터 수
    n = df.shape[0]
    gini = 1
    # 클래스마다 데이터 몇 개 있는지
    for i in list(df[label].value_counts()):
        gini -= (i / n)**2
    return gini

In [26]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [27]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    
    result=[]
    
    # 데이터 고유값 -1 만큼의 조합 생성
    for i in range(1, len(uniques)):
        # i개만큼 뽑는 경우의 수
        for j in range(len(list(combinations(uniques, i)))):
            result.append(list(list(combinations(uniques, i))[j]))
    
    return result

In [28]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [39]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    for i in binary_split:
        name = '_'.join(i) # high_low 이런 형태로 만듦
        # target과 rest 구분
        mask = np.array(list(map(lambda x : x in i, df[attribute])))
        
        target = df.loc[mask, :]
        rest = df.loc[~mask, :]        
        
        result[name] = len(target)/n*get_gini(target,label)+len(rest)/n*get_gini(rest,label)
        

    return result

In [72]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [41]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [42]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [49]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    print("Minimum Gini index of "+feature+" : "+str(round(min(get_attribute_gini_index(pd_data, feature, label).items())[1],4)))

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [50]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [51]:
len(pd_data)

14

In [54]:
from math import log2

def getEntropy(df, feature) :
    
    
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    entropy = 0
    # 유니크한 값 리스트
    for i in list(df[feature].unique()):
        # p : i 값을 지닌 데이터 개수 / 총 데이터 개수
        p = len(df[df[feature]==i])/len(df)
        entropy -= p*np.log2(p)
    return entropy


In [151]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [61]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    result = {}
    # 목표 변수 엔트로피
    info_D = getEntropy(df, feature)
    # 목표 변수 제외한 컬럼
    columns = [x for x in list(df.columns) if x != feature]
    
    for i in columns:
        w=0
        #df[i] 내 유니크한 값 리스트
        for j in list(df[i].unique()):
            w+=len(df[df[i]==j])/len(df)*getEntropy(df[df[i]==j],feature)
        result[i] = info_D - w
    return result

In [21]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443922,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [63]:
df1 = pd_data[pd_data['age'] == 'middle_aged']
df2 = pd_data[pd_data['age'] != 'middle_aged']


In [65]:
rest_features = ['income', 'student', 'credit_rating']

for feature in rest_features:
    print("middle_aged,", "Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(df1, feature, label).values()), 4))
print()

for feature in rest_features:
    print("not middle_aged,", "Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(df2, feature, label).values()), 4))

middle_aged, Minimum Gini index of income : 0.0
middle_aged, Minimum Gini index of student : 0.0
middle_aged, Minimum Gini index of credit_rating : 0.0

not middle_aged, Minimum Gini index of income : 0.375
not middle_aged, Minimum Gini index of student : 0.32
not middle_aged, Minimum Gini index of credit_rating : 0.4167


### middle_aged : 전부 다 0
### not middle_aged : 가장 작은 student, 0.32